In [11]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [12]:
class BinaryAutoencoder(nn.Module):
    def __init__(self, input_dim=(42, 22), latent_dim=10):
        super(BinaryAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim[0] * input_dim[1], 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim[0] * input_dim[1]),
            nn.Sigmoid(),  # 用 Sigmoid 将输出压缩到 [0, 1] 范围
            nn.Unflatten(1, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [23]:
def evaluate_autoencoder(model, input_data, threshold=0.5):
    model.eval()
    with torch.no_grad():
        # 生成模型输出
        output = model(input_data)
        
        # 将输出二值化
        binary_output = (output >= threshold).float()
    
    # 计算均方误差（MSE）作为相似性指标
    mse = torch.mean((binary_output - input_data) ** 2)
    
    return binary_output, mse.item()

In [7]:
X_list = []
for i in range(10):
    X = pd.read_csv(f'../VGAE_dataset/raw/X/X_{i}.csv', header=None).values
    X_list.append(X)

In [31]:
# 示例使用
X_tensor = torch.tensor(np.array(X_list)).float()

input_data_sample = X_tensor[:10]  # 获取前5个样本进行评估

# 加载模型参数
model = BinaryAutoencoder()
model_path = "binary_autoencoder.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # 切换模型到评估模式


output_data, mse_value = evaluate_autoencoder(model, input_data_sample)

# 打印输出和相似性指标
print("原始输入矩阵：\n", input_data_sample.numpy())
print("模型输出矩阵：\n", output_data.numpy())
print(f"均方误差 (MSE): {mse_value}")

/var/folders/8x/k_m4pmb11437ktb_r6tjzt2c0000gn/T/ipykernel_63377/1118011572.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))

RuntimeError: Error(s) in loading state_dict for BinaryAutoencoder:
	Missing key(s) in state_dict: "encoder.3.weight", "encoder.3.bias", "decoder.2.weight", "decoder.2.bias". 
	Unexpected key(s) in state_dict: "encoder.4.weight", "encoder.4.bias", "decoder.3.weight", "decoder.3.bias". 
	size mismatch for encoder.1.weight: copying a param with shape torch.Size([256, 924]) from checkpoint, the shape in current model is torch.Size([128, 924]).
	size mismatch for encoder.1.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for decoder.0.weight: copying a param with shape torch.Size([256, 10]) from checkpoint, the shape in current model is torch.Size([128, 10]).
	size mismatch for decoder.0.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([128]).

In [26]:
input_data_sample.numpy()[0][0,:]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1.], dtype=float32)

In [27]:
output_data.numpy()[0][0,:]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1.], dtype=float32)

In [30]:
index=2
np.array_equal(input_data_sample.numpy()[index], output_data.numpy()[index])

IndexError: index 2 is out of bounds for axis 0 with size 2